# Libro para procesar el audio del podcast | Book for processing podcast audio

Este libro permite editar el audio de un podcast y partirlo en sus silencios, adicionar segementos, subir y bajar volumen, entre otros.


This book allows you to edit the audio of a podcast and split it into its silences, add segments, increase and decrease volume, among others.

In [ ]:
# Parámetros iniciales
episode = "T0E0"

file_path = f"inputs/{episode}/{episode}.mp3"
context_path = f"inputs/{episode}/{episode}-Context.mp3"
presentation_path = f"inputs/{episode}/{episode}-Presentation.mp3"
close_path = f"inputs/{episode}/{episode}-Close.mp3"
output_path = f"outputs/{episode}.mp3"

episode_folder = "00-Piloto"
assets_folder = 'assets'
base = '.'

## Validación de variables

In [ ]:
# Checa si el libro está corriendo en Google Colab
import os

def is_running_in_colab():
  """Checks if the code is running in Google Colab."""
  try:
    from google.colab import drive
    return True
  except ImportError:
    return False

if is_running_in_colab():
  print("This notebook is running in Google Colab.")

  # Activar unidad Drive
  from google.colab import drive
  drive.mount('/content/drive')
  base = f"/content/drive/MyDrive/Podcast/{episode_folder}"
  file_path = f"{base}/{file_path}"
  context_path = f"{base}/{context_path}"
  presentation_path = f"{base}/{presentation_path}"
  close_path = f"{base}/{close_path}"
  assets_folder = f"/content/drive/MyDrive/Editor/{assets_folder}"
  output_path = f"{base}/{output_path}"

  import sys
  # Add the directory containing your module to the Python path
  sys.path.append('/content/drive/MyDrive/Editor')

In [ ]:
# Verificar que los archivos existan antes de continuar
import os
if not os.path.exists(file_path):
  raise FileNotFoundError(f"File not found: {file_path}")
if not os.path.exists(context_path):
  raise FileNotFoundError(f"File not found: {context_path}")
if not os.path.exists(presentation_path):
  raise FileNotFoundError(f"File not found: {presentation_path}")
if not os.path.exists(close_path):
  raise FileNotFoundError(f"File not found: {close_path}")
if not os.path.exists(assets_folder):
  raise FileNotFoundError(f"File not found: {assets_folder}")

In [ ]:
# Si no existe la carpeta de output_path la crea
import os
# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

## Prepración del libro

In [ ]:
! pip install pydub scipy
import zds_utils as zds
from pydub import AudioSegment, silence

In [ ]:
# Tracks precargados
INTRO_ZDS = AudioSegment.from_file(f"{assets_folder}/Intro-ZDS.mp3")
ENDING = AudioSegment.from_file(f"{assets_folder}/Ending.mp3")
INTENSE = AudioSegment.from_file(f"{assets_folder}/Intense.mp3")
SHORT_INTENSE = AudioSegment.from_file(f"{assets_folder}/Short-Intense.mp3")
PIANO = AudioSegment.from_file(f"{assets_folder}/Piano.mp3")
FUTURE = AudioSegment.from_file(f"{assets_folder}/Future.mp3")

In [ ]:
# Variables de configuración
silence_threshold = -40   # Umbral de silencio en dBFS
min_silence_length = 3000  # Longitud mínima de silencio en ms

## Preparación del audio

In [ ]:
# Carga los segmentos de audio y de silencios
chunks = zds.chunks_load(f"{base}/inputs/{episode}/chunks")
silences = zds.silences_load(f"{base}/inputs/{episode}/silences.txt")
audio = []

if len(chunks) == 0 or len(silences) == 0:
  print("No se encontraron segmentos de audio o silencios guardados, se procederá a generarlos del audio original")
  # Carga del archivo de audio
  audio = AudioSegment.from_file(file_path)

  if len(chunks) == 0:
    # Divide el audio en fragmentos de audio en base al silencio detectado
    print("Dividiendo el audio en fragmentos de audio...")
    chunks = silence.split_on_silence(audio, min_silence_len=min_silence_length, silence_thresh=silence_threshold)
    print(f"Se han detectado {len(chunks)} fragmentos de audio. Se procederá a guardarlos.")
    zds.chunks_save(chunks, f"{base}/inputs/{episode}/chunks")
    print(f"Se han guardado los fragmentos de audio en {base}/inputs/{episode}/chunks")

  if len(silences) == 0:
    # Extraer los fragmentos de silencio que superen una duración mínima
    print("Extrayendo los fragmentos de silencio...")
    silences = silence.detect_silence(audio, min_silence_len=min_silence_length, silence_thresh=silence_threshold)
    print(f"Se han detectado {len(silences)} fragmentos de silencio. Se procederá a guardarlos.")
    zds.silences_save(silences, f"{base}/inputs/{episode}/silences.txt")
    print(f"Se han guardado los fragmentos de silencio en {base}/inputs/{episode}/silences.txt")

print(f"Se han cargado {len(chunks)} fragmentos de audio y {len(silences)} fragmentos de silencio")

## Procesamiento

In [ ]:
# Estadísticas del audio
print(f"Tiempo total del audio:     {zds.ms_to_time(len(audio))}")
print(f"Cantidad de chunks:         {len(chunks)}")
print(f"Duración de los Chunks:     {zds.ms_to_time(sum([len(chunk) for chunk in chunks]))}")
#print(f"Duración de cada chunk:    {[zds.ms_to_time(len(chunk)) for chunk in chunks]}")
print(f"Cantidad de silencios:      {len(silences)}")
print(f"Duración de los silencios:  {zds.ms_to_time(sum([end - start for start, end in silences]))}")
#print(f"Duración de cada silencio: {[zds.ms_to_time(end - start) for start, end in silences]}")

In [ ]:
# Tiempos del audio intercalando chuks y silencios
chunks_durations = [len(chunk) for chunk in chunks]
silence_durations = [(start, end, end - start) for start, end in silences]
for chunk, silence in zip(chunks_durations, silence_durations):
  print(f"Silence: {zds.ms_to_time(silence[0],True)}-{zds.ms_to_time(silence[1],True)}->{zds.ms_to_time(silence[2],True)}")
  print(f"Chunk:   {zds.ms_to_time(silence[1],True)}-{zds.ms_to_time(silence[1]+chunk,True)}->{zds.ms_to_time(chunk,True)}")

In [ ]:
# Inserta traccks en algunas posiciones de la lista de chunks
chunks_final = chunks.copy()
special_tracks = {
  0: SHORT_INTENSE,
  4: INTENSE,
  13: PIANO
}
# Agregar silencio en los lugares donde NO se insertan los tracks
silence = AudioSegment.silent(duration=1000)
inserted = 0
for i in range(len(chunks_final)):
  if i in special_tracks:
    chunks_final.insert(i + inserted, special_tracks[i])
  else:
    chunks_final.insert(i + inserted, silence)
  inserted += 1

chunks_final.insert(-1, FUTURE)

In [ ]:
# Une los fragmentos para la estructura final del audio
context = AudioSegment.from_file(context_path)
context = context.fade_in(1000).fade_out(2000)
# Otros audios auxiliares
presentation = AudioSegment.from_file(presentation_path)
close = AudioSegment.from_file(close_path)

# Compila el audio final
compiled_audio = context + INTRO_ZDS + presentation + sum(chunks_final) + close + ENDING
#compiled_audio = context + INTRO_ZDS + sum(chunks_final) + ENDING
print(f"Duración del audio compilado: {zds.ms_to_time(len(compiled_audio))}")

In [ ]:
# Guardar el archivo de audio Final
zds.audio_save(compiled_audio, f"outputs/{episode}.mp3", verbose=True)
#zds.audio_save(chunks_final[:30], f"outputs/{episode}.mp3", verbose=True)
#videos para yorsh sobre educación e ia

In [ ]:
output_path